# Задание 8
<ol>
<li>Прочитать excel  в df</li>
<li>Удалить компании статус которых ‘Liquidating’ или ‘Out of Business’</li>
<li>Создать два df</li>
    <ul>
    <li>Колонки для первого: Company Name, Product Name,  Product Description, Company Status,  Industry Classifications</li>
    <li>Колонки для второго: Company Name, SIC Codes</li> 
    <li>*остальное нам не нужно</li>
    </ul>
<li>Парсинг полей:
    <ol>
    <li>Для первого разделить
        <ul>
        <li>по ‘;’
            <ol>
        <li>    Product Name</li>
        <li>    Industry Classifications</li>
            </ol></li>
        <li>По ‘\n\n’
            <ul>
                <li>
        :    Product Description
                    </li>
</ul>
                </li> 
        </ul>   
    </li>
    <li>
    Для второго:
        <ul>
        <li>Сначала объединить все коды: Для каждой компании “SIC Codes” - объединяем значения, оставляем уникальные (напр. SIC коды: (0001,0002)+(0001,0004,0005) -> (0001, 0002, 0004, 0005))</li>
        <li>Затем добавить новую колонку с разделенными sic codes по ‘;’</li>
        </ul>    
    </li>    
        </ol>
</li>
<li>Убрать дубли для каждого df</li>
<li>Записать оба df в csv
    <ul>
    <li>Название для первого - Products</li>
    <li>Для второго - Sic_Codes</li>
    </ul>
</li>
</ol>

In [1]:
import os
import shutil 
import numpy as np
import pandas as pd
from datetime import datetime
import xlrd
import re
from itertools import chain

In [2]:
file_name = 'Задание 8.xls'
file_sheet_name = 'Screening'
df = pd.read_excel(file_name, file_sheet_name, header = 7)

In [3]:
df = df.loc[~df['Company Status'].isin(['Liquidating','Out of Business'])]

In [4]:

df = df.reset_index(drop=True)

In [5]:
df1 = df.loc[:,['Company Name', 'Product Name', 'Product Description', 'Company Status', 'Industry Classifications']]
df2 = df.loc[:,['Company Name', 'SIC Codes']]

In [6]:
# return list from series of simbol separeted strings
def chainer(s,split_simbol):
    return list(chain.from_iterable(s.str.split(str(split_simbol))))


In [7]:
# calculate lengths of splits
lens = df1['Product Name'].str.split(';').map(len)
# create new dataframe, repeating or chaining as appropriate
df1 = pd.DataFrame({'Company Name': np.repeat(df1['Company Name'], lens),
                    'Product Name': chainer(df1['Product Name'],';'),
                    'Product Description': np.repeat(df1['Product Description'], lens),
                    'Company Status': np.repeat(df1['Company Status'], lens),
                    'Industry Classifications': np.repeat(df1['Industry Classifications'], lens)})

In [8]:
# calculate lengths of splits
lens = df1['Industry Classifications'].str.split(';').map(len)
# create new dataframe, repeating or chaining as appropriate
df1 = pd.DataFrame({'Company Name': np.repeat(df1['Company Name'], lens),
                    'Product Name': np.repeat(df1['Product Name'], lens),
                    'Product Description': np.repeat(df1['Product Description'], lens),
                    'Company Status': np.repeat(df1['Company Status'], lens),
                    'Industry Classifications': chainer(df1['Industry Classifications'],';')})

In [9]:
# calculate lengths of splits
lens = df1['Product Description'].str.split('\n\n').map(len)
# create new dataframe, repeating or chaining as appropriate
df1 = pd.DataFrame({'Company Name': np.repeat(df1['Company Name'], lens),
                    'Product Name': np.repeat(df1['Product Name'],lens),
                    'Product Description': chainer(df1['Product Description'], '\n\n'),
                    'Company Status': np.repeat(df1['Company Status'], lens),
                    'Industry Classifications': np.repeat(df1['Industry Classifications'], lens)})

In [10]:
for i in range(len(df2)):
    temporary_variavle = str(df2.loc[i,'SIC Codes'])
    temporary_variavle = set(temporary_variavle.split(';'))
    temporary_variavle = str(temporary_variavle).replace('{','').replace('}','').replace(r"'",'').replace(',',';')
    df2.loc[i,'SIC Codes'] = temporary_variavle
    df2.loc[i,'SIC CODES with ; ssepareted'] = temporary_variavle

In [11]:
df1.drop_duplicates(inplace=True)
df2.drop_duplicates(inplace=True)

In [13]:
df2.loc[40:50,:]

,Company Name,SIC Codes,SIC CODES with ; ssepareted
40,Vk Holding A/S,6719 Holding companies,6719 Holding companies
41,"Vk Komplekt, Ooo",5039 Construction materials ; 5199 Nondurable...,5039 Construction materials ; 5199 Nondurable...
42,Vk System AB,5063 Electrical apparatus and equipment,5063 Electrical apparatus and equipment
43,"Vk Tekhproekt, Tov",5039 Construction materials ; 5199 Nondurable...,5039 Construction materials ; 5199 Nondurable...
44,"Vk Telecom Material Co., Ltd.",3691 Storage batteries,3691 Storage batteries
45,VK Union Corporation,5541 Gasoline service stations,5541 Gasoline service stations
46,Vk&Sb Pty. Ltd.,5541 Gasoline service stations,5541 Gasoline service stations
47,Vk-Electric Oy,-,-
48,"Vk-Electric Oy, Distribution Network Contract...",-,-
49,"Vk-Inzhiniring, Ooo",1629 Heavy construction,1629 Heavy construction


In [12]:
df1.to_csv('Products.csv',index=False)
df2.to_csv('Sic_Codes.csv',index=False)